<a href="https://colab.research.google.com/github/peter-yanase/ndlocr_remix/blob/main/ndlocr_in_colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRの実行例： 最小限のリソースでOCRを実行する

## 開発目的

- 各サーバーへの負担を軽減すること
- 立ち上げに必要な時間を大幅に短縮化すること
- ndlocr_cliを通さず、各推論処理を別々に実行すること
- GPUが必要とそうでないプロセスを切り分けること
- ndlocrの仕組を明確にすること

## 実行例

### Google Driveをマウントする

**必須のステップ**

In [ ]:
from pathlib import Path
from google.colab import drive

gdrive: Path = Path().cwd() / 'gdrive'
drive.mount(str(gdrive))

ndlocr: Path = gdrive / 'MyDrive' / 'ndlocr'

### separate_pages_ssd

In [ ]:
separate_pages_ssd = ndlocr / 'separate_pages_ssd'

#### インストール

必要なファイルをGoogle Driveにコピーすることによって、各サーバーの負担を減らし、かつ立ち上げを高速化する。

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/separate_pages_ssd/master/inference_divided.py -P {separate_pages_ssd}
!wget -nc https://raw.githubusercontent.com/ndl-lab/separate_pages_ssd/master/ssd_tools/ssd_layers.py -P {ssd_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/separate_pages_ssd/master/ssd_tools/ssd.py -P {ssd_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/separate_pages_ssd/master/ssd_tools/ssd_utils.py -P {ssd_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/separate_pages_ssd/master/ssd_tools/dpiconfig.json -P {ssd_tools}

In [ ]:
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P {ssd_tools}

#### 実行

GPUはなくてもいい

In [ ]:
%cd {separate_pages_ssd}

使い方の説明：　https://github.com/ndl-lab/separate_pages_ssd#%E4%BD%BF%E3%81%84%E6%96%B9

In [ ]:
# テスト用サンプル画像
# -----------------
#!wget -nc https://dl.ndl.go.jp/api/iiif/799179/R0000036/full/full/0/default.jpg -P inference_input

In [ ]:
# テスト用画像のオプション設定
# そのままで
# --------
#!python inference_divided.py

### deskew_HT

In [ ]:
deskew_HT = ndlocr / 'deskew_HT'

#### インストール

必要なファイルをGoogle Driveにコピーすることによって、各サーバーの負担を減らし、かつ立ち上げを高速化する。

In [ ]:
alyn3 = deskew_HT / 'alyn3'

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/deskew_HT/master/alyn3/deskew.py -P {alyn3}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deskew_HT/master/alyn3/skew_detect.py -P {alyn3}

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/deskew_HT/master/run_deskew.py -P {deskew_HT}

#### 実行

GPUは不要

In [ ]:
%cd {deskew_HT}

In [ ]:
!python run_deskew.py

使い方の説明：　https://github.com/ndl-lab/deskew_HT#%E4%BD%BF%E3%81%84%E6%96%B9

In [ ]:
# テスト用サンプル画像
# -----------------
#!wget -nc https://raw.githubusercontent.com/kakul/Alyn/master/examples/4.jpg -P deskew_input

In [ ]:
# テスト用画像のオプション設定
#　画像のフォルダーを指定し、精度と修正の最大値を上げた
# --------------------------------------------
#!python run_deskew.py \
#    deskew_input \
#    -a 0.1 \
#    -s 15

### ndl_layout

In [ ]:
ndl_layout = ndlocr / 'ndl_layout'
layout_models = ndl_layout / 'models'
mmcv = ndl_layout / 'mmcv'

#### インストール

必要なファイルをGoogle Driveにコピーすることによって、各サーバーの負担を減らし、かつ立ち上げを高速化する。

In [ ]:
layout_tools = ndl_layout / 'tools'

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/ndl_layout/master/tools/process.py -P {layout_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/ndl_layout/master/tools/utils.py -P {layout_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/ndl_layout/master/tools/ndl_parser.py -P {layout_tools}
!wget -nc https://raw.githubusercontent.com/ndl-lab/ndl_layout/master/models/ndl_layout_config.py -P {layout_models}

In [ ]:
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P {layout_models}

In [ ]:
!wget -nc https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/mmcv-2.0.0-cp310-cp310-manylinux1_x86_64.whl -P {mmcv}

## 実行

GPUは必須

In [ ]:
mmcv_wheel = [file for file in mmcv.iterdir()][0]

!pip install {mmcv_wheel} mmdet==3.0.0

In [ ]:
%cd {ndl_layout}

In [ ]:
!python -m tools.process \
  --config ./models/ndl_layout_config.py \
  --checkpoint ./models/epoch_140_all_eql_bt.pth

使い方の説明：　https://github.com/ndl-lab/ndl_layout#%E4%BD%BF%E3%81%84%E6%96%B9

In [ ]:
# テスト用サンプル画像
#!wget -nc http://dglb01.ninjal.ac.jp/ninjaldl/meirokuzassi/007/jpg/mrzs007-020.jpg -P input/img

In [ ]:
# テスト用画像のオプション設定
#　画像のフォルダーを指定した
#　メモ：出力のパスはファイル名まで指定する必要がある。また、既存のフォルダーしか指定できない。
# -----------------------------------------------------------------------------
#!python -m tools.process \
#  --img_paths input/img/*.jpg \
#  --output_path input/xml/layout.xml \
#  --config models/ndl_layout_config.py \
#  --checkpoint models/epoch_140_all_eql_bt.pth

## text_recognition


In [ ]:
text_recognition = ndlocr / 'text_recognition'

#### インストール

必要なファイルをGoogle Driveにコピーすることによって、各サーバーの負担を減らし、かつ立ち上げを高速化する。

In [ ]:
text_models = text_recognition / 'models'
deeptext_modules = text_recognition / 'modules'

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/text_recognition/master/text_recognition.py -P {text_recognition}
!wget -nc https://raw.githubusercontent.com/ndl-lab/text_recognition/master/xmlparser.py -P {text_recognition}

In [ ]:
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P {text_models}
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P {text_models}

In [ ]:
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/utils.py -P {text_recognition}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/dataset.py -P {text_recognition}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/model.py -P {text_recognition}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/modules/feature_extraction.py -P {deeptext_modules}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/modules/prediction.py -P {deeptext_modules}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/modules/sequence_modeling.py -P {deeptext_modules}
!wget -nc https://raw.githubusercontent.com/ndl-lab/deep-text-recognition-benchmark/97e546b4e02855299e64398ac4f216668500fa17/modules/transformation.py -P {deeptext_modules}

## 実行

GPUはあった方が早い

In [ ]:
!pip install lmdb

In [ ]:
%cd {text_recognition}

In [ ]:
!python text_recognition.py

使い方の説明：　https://github.com/ndl-lab/text_recognition#%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E8%AA%AC%E6%98%8E3

In [ ]:
# テスト用ファイルに関して
# text_recognitionはndl_layoutで作ったxmlに基づいて処理を行うので、
# まずndl_layoutのテストを実行して下さい。

In [ ]:
#input = ndl_layout / 'input'

In [ ]:
# ドキュメンテーションは少し説明不足なので、以下の設定を参照して下さい。
# ---------------------------------------------------------
#!python text_recognition.py \
#    --saved_model models/ndlenfixed64-mj0-synth1.pth \
#    --character "〓$(cat models/mojilist_NDL.txt | tr -d '\n')" \
#    --imgW 1200 \
#    --workers -1 \
#    --sensitive \
#    --xml results \
#    --FeatureExtraction ResNet \
#    --Prediction CTC \
#    --PAD \
#    --SequenceModeling None \
#    --Transformation None \
#    --db_path {input}